# Replication: Financial Intermediation, International Risk Sharing, and Reserve Currencies

## Autarky

In [1]:
mutable struct GertlerKiyotakiModel
    
    # utility function 
    ρ::Float64
    
    # stochastic process dY/Y
    μ::Float64
    σ::Float64

    # switching from financier to saver
    λ::Float64
    β::Float64
    δ::Float64
    
end

In [2]:
function GertlerKiyotakiModel(; ρ = 0.01, μ = 0.01, σ = 0.05, λ = 0.014, β = 1/3, δ = 0.004)
    GertlerKiyotakiModel(ρ, μ, σ, λ, β, δ)
end

GertlerKiyotakiModel

In [3]:
function initialize_state(m::GertlerKiyotakiModel; N_grid_length = 100)
    OrderedDict(:N => linspace(0.00001, 100., N_grid_length))
end

function initialize_y(m::GertlerKiyotakiModel, state)
    OrderedDict(:Q => fill(0.1, length(state[:N])), 
        :Ω => fill(0.1, length(state[:N])))
end

initialize_y (generic function with 1 method)

In [10]:
function (m::GertlerKiyotakiModel)(state, y)
    #unpack constants, state, current solution
    ρ = m.ρ; μ = m.μ; σ = m.σ; λ = m.λ; β = m.β; δ = m.δ
    N = state.N
    Q = y.Q; QN = y.QN; QNN = y.QNN; Ω = y.Ω; ΩN = y.ΩN; ΩNN = y.ΩNN;
    
    # interest rate
    rd = ρ + μ - σ^2
    
    # (dQ+Y)/Q
    μQ = (1 + Q * (μ + QN * (δ - μ - ρ)) + N * QN * (ρ - λ)) / (Q * (1 - QN)) + 
        (Q - N) * QN * σ^2 / (Q * (1 - QN)) + (Q - N)^2 * QNN * σ^2 / (2 * Q * (1 - QN)^3)
    σQ = (Q - N * QN) / (Q * (1 - QN)) * σ
    
    # dN/N process
    μN = rd - λ - μ + σ^2 + Q/N * (μQ - rd + δ - σ * σQ)
    σN = Q/N * σQ - σ
    
    # dΩ/Ω
    μΩ = ΩN * N / Ω * μN + 1/2 * ΩNN * N^2 / Ω * σN^2
    σΩ = ΩN * N / Ω * σN
    
    # PDE
    Qt = μQ - rd - σ * σQ + σΩ * σQ
    Ωt = λ * (1 - Ω) / Ω + μΩ - σ * σΩ
    
    return (Qt, Ωt), μN, tuple(:Q => Q, :Ω => Ω, :rd => rd)
end

In [13]:
using EconPDEs

m = GertlerKiyotakiModel()
state = initialize_state(m)
y0 = initialize_y(m, state)
y, result, distance = pdesolve(m, state, y0, iterations = 24, 
    inner_iterations = 100, Δ = 1e-5)

(iter, Δ, distance) = (1, 1.0e-5, 120.43163196093421)
(iter, Δ, distance) = (2, 6.63850066403483e-5, 15.023346878215715)
(iter, Δ, distance) = (3, 0.0021286480974561984, 1.923896641097308)
(iter, Δ, distance) = (4, 0.1329776998061547, 0.21881894034819283)
(iter, Δ, NaN) = (5, 18.70663295166771, NaN)
(iter, Δ, NaN) = (6, 1.870663295166771, NaN)
(iter, Δ, distance) = (7, 0.1870663295166771, 0.11825203174796696)
(iter, Δ, NaN) = (8, 0.6923120963690148, NaN)
(iter, Δ, distance) = (9, 0.06923120963690148, 0.10206073255531276)
(iter, Δ, distance) = (10, 0.1604286192144693, 0.08209232527260088)
(iter, Δ, NaN) = (11, 0.7978072174467137, NaN)
(iter, Δ, distance) = (12, 0.07978072174467137, 0.07533083048372032)
(iter, Δ, distance) = (13, 0.17388325385213443, 0.06550327681420479)
(iter, Δ, NaN) = (14, 0.7998848642058957, NaN)
(iter, Δ, distance) = (15, 0.07998848642058956, 0.0620351796003385)
(iter, Δ, distance) = (16, 0.16892053836912077, 0.05649773143187545)
(iter, Δ, NaN) = (17, 0.741907023190

(DataStructures.OrderedDict(:Q=>[-9.71939, -8.93914, -8.28668, -7.71075, -7.18361, -6.69712, -6.23808, -5.80957, -5.40601, -5.0286  …  -0.0588945, -0.0553666, -0.0522813, -0.0483196, -0.0471667, -0.039275, -0.0470138, -0.0243069, -0.0509618, -0.0198893],:Ω=>[0.000169625, 0.000829109, 0.00291036, 0.00657637, 0.0112253, 0.0160271, 0.0211783, 0.0259647, 0.0308164, 0.0353125  …  0.0979936, 0.0980408, 0.098074, 0.0981361, 0.0981271, 0.0982689, 0.098105, 0.0984808, 0.098052, 0.0985043]), DataStructures.OrderedDict(:Q=>[-9.71939, -8.93914, -8.28668, -7.71075, -7.18361, -6.69712, -6.23808, -5.80957, -5.40601, -5.0286  …  -0.0588945, -0.0553666, -0.0522813, -0.0483196, -0.0471667, -0.039275, -0.0470138, -0.0243069, -0.0509618, -0.0198893],:Ω=>[0.000169625, 0.000829109, 0.00291036, 0.00657637, 0.0112253, 0.0160271, 0.0211783, 0.0259647, 0.0308164, 0.0353125  …  0.0979936, 0.0980408, 0.098074, 0.0981361, 0.0981271, 0.0982689, 0.098105, 0.0984808, 0.098052, 0.0985043],:rd=>[0.0175, 0.0175, 0.0175,

That is not a meaningful solution!

The explicit formulation in the appendix is not working either:

In [12]:
function (m::GertlerKiyotakiModel)(state, y)
    #unpack constants, state, current solution
    ρ = m.ρ; μ = m.μ; σ = m.σ; λ = m.λ; β = m.β; δ = m.δ
    N = state.N
    Q = y.Q; QN = y.QN; QNN = y.QNN; Ω = y.Ω; ΩN = y.ΩN; ΩNN = y.ΩNN;
    
    # interest rate
    rd = ρ + μ - σ^2
    
    # (dQ+Y)/Q
    μQ = (1 + Q * (μ + QN * (δ - μ - ρ)) + N * QN * (ρ - λ)) / (Q * (1 - QN)) + 
        (Q - N) * QN * σ^2 / (Q * (1 - QN)) + (Q - N)^2 * QNN * σ^2 / (2 * Q * (1 - QN)^3)
    σQ = (Q - N * QN) / (Q * (1 - QN)) * σ
    
    # dN/N process
    μN = rd - λ - μ + σ^2 + Q/N * (μQ - rd + δ - σ * σQ)
    
    # PDE
    Qt = 2 * (-1 + QN) * (
        - (-1 + QN) * (1 + Q * (QN * δ - ρ) + N * QN * (-λ + ρ)) * Ω + (-N + Q) * (Q - N * QN) * σ^2 * ΩN
        ) / (
        (N - Q)^2 * σ^2 * Ω
        ) - QNN
    Ωt = (
        2 * (-1 + QN)^2 * λ * Ω * (-1 + Ω + N * ΩN) + 
        2 * ΩN * (- (-1 + QN) * ((-1 + QN) * (Q * δ + N * ρ) + (-N + Q) * σ^2) * Ω)
        ) / (
        (N - Q)^2 * σ^2 * Ω
        ) +
        (2 * ΩN * (-N + Q) * (Q - N * QN) * σ^2 * ΩN) / ((N - Q)^2 * σ^2 * Ω) - ΩNN
    
    return (Qt, Ωt), μN, tuple(:Q => Q, :Ω => Ω, :rd => rd)
end

In [14]:
y, result, distance = pdesolve(m, state, y0, iterations = 24, 
    inner_iterations = 100, Δ = 1e-5)

(iter, Δ, distance) = (1, 1.0e-5, 120.43163196093421)
(iter, Δ, distance) = (2, 6.63850066403483e-5, 15.023346878215715)
(iter, Δ, distance) = (3, 0.0021286480974561984, 1.923896641097308)
(iter, Δ, distance) = (4, 0.1329776998061547, 0.21881894034819283)
(iter, Δ, NaN) = (5, 18.70663295166771, NaN)
(iter, Δ, NaN) = (6, 1.870663295166771, NaN)
(iter, Δ, distance) = (7, 0.1870663295166771, 0.11825203174796696)
(iter, Δ, NaN) = (8, 0.6923120963690148, NaN)
(iter, Δ, distance) = (9, 0.06923120963690148, 0.10206073255531276)
(iter, Δ, distance) = (10, 0.1604286192144693, 0.08209232527260088)
(iter, Δ, NaN) = (11, 0.7978072174467137, NaN)
(iter, Δ, distance) = (12, 0.07978072174467137, 0.07533083048372032)
(iter, Δ, distance) = (13, 0.17388325385213443, 0.06550327681420479)
(iter, Δ, NaN) = (14, 0.7998848642058957, NaN)
(iter, Δ, distance) = (15, 0.07998848642058956, 0.0620351796003385)
(iter, Δ, distance) = (16, 0.16892053836912077, 0.05649773143187545)
(iter, Δ, NaN) = (17, 0.741907023190

(DataStructures.OrderedDict(:Q=>[-9.71939, -8.93914, -8.28668, -7.71075, -7.18361, -6.69712, -6.23808, -5.80957, -5.40601, -5.0286  …  -0.0588945, -0.0553666, -0.0522813, -0.0483196, -0.0471667, -0.039275, -0.0470138, -0.0243069, -0.0509618, -0.0198893],:Ω=>[0.000169625, 0.000829109, 0.00291036, 0.00657637, 0.0112253, 0.0160271, 0.0211783, 0.0259647, 0.0308164, 0.0353125  …  0.0979936, 0.0980408, 0.098074, 0.0981361, 0.0981271, 0.0982689, 0.098105, 0.0984808, 0.098052, 0.0985043]), DataStructures.OrderedDict(:Q=>[-9.71939, -8.93914, -8.28668, -7.71075, -7.18361, -6.69712, -6.23808, -5.80957, -5.40601, -5.0286  …  -0.0588945, -0.0553666, -0.0522813, -0.0483196, -0.0471667, -0.039275, -0.0470138, -0.0243069, -0.0509618, -0.0198893],:Ω=>[0.000169625, 0.000829109, 0.00291036, 0.00657637, 0.0112253, 0.0160271, 0.0211783, 0.0259647, 0.0308164, 0.0353125  …  0.0979936, 0.0980408, 0.098074, 0.0981361, 0.0981271, 0.0982689, 0.098105, 0.0984808, 0.098052, 0.0985043],:rd=>[0.0175, 0.0175, 0.0175,

## Open Banking Economy with a single Tree

In [ ]:
# 
mutable struct MaggioriSingleTreeModel
    
    # utility function 
    ρ::Float64
    
    # stochastic process dY/Y
    μ::Float64
    σ::Float64

    # switching from financier to saver
    λ::Float64
    β::Float64
    δ::Float64
    
end

function MaggioriSingleTreeModel(; ρ = 0.01, μ = 0.01, σ = 0.05, λ = 0.014, β = 1/3, δ = 0.004)
    MaggioriSingleTreeModel(ρ, μ, σ, λ, β, δ)
end

function initialize_state(m::MaggioriSingleTreeModel; Nstar_grid_length = 100)
    OrderedDict(:Nstar => linspace(0.00001, 100., Nstar_grid_length))
end

function initialize_y(m::MaggioriSingleTreeModel, state)
    OrderedDict(:Q => fill(0.1, length(state[:Nstar])), 
        :Ωstar => fill(0.1, length(state[:Nstar])))
end

In [ ]:
function (m::MaggioriSingleTreeModel)(state, y)
    #unpack constants, state, current solution
    ρ = m.ρ; μ = m.μ; σ = m.σ; λ = m.λ; β = m.β; δ = m.δ
    Nstar = state.Nstar
    Q = y.Q; QNstar = y.QNstar; QNstarNstar = y.QNstarNstar; 
    Ωstar = y.Ωstar; ΩstarNstar = y.ΩstarNstar; ΩstarNstarNstar = y.ΩstarNstarNstar;
    
    # interest rate
    rd = ρ + μ - σ^2
    
    # (dQ+Y)/Q
    μQ = (1 + Q * (μ + QN * (δ - μ - ρ)) + N * QN * (ρ - λ)) / (Q * (1 - QN)) + 
        (Q - N) * QN * σ^2 / (Q * (1 - QN)) + (Q - N)^2 * QNN * σ^2 / (2 * Q * (1 - QN)^3)
    σQ = (Q - N * QN) / (Q * (1 - QN)) * σ
    
    # dN/N process
    μN = rd - λ - μ + σ^2 + Q/N * (μQ - rd + δ - σ * σQ)
    σN = Q/N * σQ - σ
    
    # dΩ/Ω
    μΩ = ΩN * N / Ω * μN + 1/2 * ΩNN * N^2 / Ω * σN^2
    σΩ = ΩN * N / Ω * σN
    
    # PDE
    Qt = μQ - rd - σC * σQ
    Ωt = μΩstar - σCstar * σΩstar
    
    return (Qt, Ωt), μN, tuple(:Q => Q, :Ω => Ω, :rd => rd)
end

